## DATA CLEANING AND EDA cont.

In [1]:
# import required packages
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import missingno as mn

%matplotlib inline
%config InlineBackend.figure_format = 'svg'
plt.style.use("seaborn")

In [2]:
df = pd.read_pickle("project2_data/SFproperty_df")

In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1182 entries, 0 to 1181
Data columns (total 14 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   Address     1182 non-null   object
 1   Location    1182 non-null   object
 2   Price       1182 non-null   object
 3   Beds        1182 non-null   object
 4   Baths       1182 non-null   object
 5   Sq.Ft.      1182 non-null   object
 6   $/Sq.Ft.    1182 non-null   object
 7   On Redfin   1182 non-null   object
 8   HOA         1182 non-null   object
 9   Year Built  1182 non-null   object
 10  Lot Size    1182 non-null   object
 11  Date Sold   1182 non-null   object
 12  Zip Code    1182 non-null   object
 13  prop_type   1182 non-null   object
dtypes: object(14)
memory usage: 129.4+ KB


### LOOKING into missing values in size

In [ ]:
df[df["Sq.Ft."].isna()].sort_values("Price", ascending = True).head()

In [ ]:
df[df["Sq.Ft."].isna()].sort_values("Price", ascending = True).tail()

In [ ]:
#can I fill in missing size with Price/($/Sq.Ft.)??
#$/Sq.Ft. has more missing values than size, cannot be use to fill in missing values in size
mask = (df["Sq.Ft."].isna()) & (df["$/Sq.Ft."] == "—")
len(df[mask])

### ^ CONCLUSION: missing values can be filled in from individual property page <br> Dropping observation with missing values (back to scraping if more time)

In [ ]:
#check other missing values
df.info()

### NOW GOING BACK TO MISSING VALUES in Beds and Baths after removing missing values in size

In [ ]:
#check missing values in Beds
print(df["Beds"].isna().sum())
df[df["Beds"].isna()].sort_values("prop_type").head()#still have 31 missing, 30 multi-family units...T.T
df[df["Beds"].isna()].sort_values("prop_type").tail()

In [ ]:
len(df[df["prop_type"] == "multi_fam"])

In [ ]:
#check missing values in Baths
print(df["Baths"].isna().sum())
df[df["Baths"].isna()].sort_values("prop_type").head(10)#still have 99 missing values, 92 multi-family units
df[df["Baths"].isna()].sort_values("prop_type").tail()

### ^ CONCLUSION: some values can be filled from indivial page, and both Beds and Baths are missing @ random (mostly missing in multi-family units) -> Safe to drop NAs

In [ ]:
#drop observations with missing values in Beds and Baths
df.dropna(subset = ["Beds", "Baths"], axis = 0, inplace = True)
df.info()

In [ ]:
#check missing values in HOA
print(len(df[df["HOA"] == ""])) #no missing value in HOA
df["HOA"].value_counts() 

#turn HOA into numeric
df["HOA"] = ["".join(h.split("/")[0].strip("$").split(",")) for h in df["HOA"]]
df["HOA"].replace({"None":"0"}, inplace = True)
df["HOA"] = pd.to_numeric(df["HOA"])
df["HOA"].value_counts()

In [ ]:
#check missing values in Year Built
print(len(df[df["Year Built"] == "—"])) #28 missing values in Year Built
df["Year Built"].replace({"—":None}, inplace = True)

#convert Year Built to an interger
df["Year Built"] = pd.to_numeric(df["Year Built"]).astype("Int64")
df["Year Built"].value_counts().sort_index(ascending = False)
df["Year Built"].isna().sum()

### LOOK at missing values in Year Built

In [ ]:
df[df["Year Built"].isna()].sort_values("Price").head(8)

In [ ]:
df[df["Year Built"].isna()].sort_values("Price").tail()

In [ ]:
#look at distribution of Year Built
YB_noNA = df[df["Year Built"].notnull()]["Year Built"]
plt.hist(YB_noNA, bins = 100)
plt.title("Property Distribution by Year Built", fontsize = 15)
plt.ylabel("Number of Properties", fontsize = 12)
plt.xlabel("Year Built", fontsize = 12);

In [ ]:
# should do this later after converting other features to the appropriate data type

plt.rcParams["figure.figsize"] = [6, 4]
mn.matrix(df[["Price", "Beds", "Baths"]])

In [ ]:
# drop records with missing values in Price and a record with Price misrecorded
print(df["Price"].isna().sum()) #13 obs missing Price
df[df["Price"].isna()]
df[df["Price"].isna()].groupby(["prop_type"])["prop_type"].count()

df.dropna(subset = ["Price"], axis = 0, inplace = True)
df.info()

### ^ CONCLUSION: Looks like missing at completely random, info. can be found in individual page <br> Impute with random sampling from the range of Year Built

In [ ]:
#number of NAs in Year Built
mask = df["Year Built"].isna()
YB_NAs = mask.sum()

#fill in NAs with random sampling
fill = np.random.choice(df[df["Year Built"].notnull()]["Year Built"].values, size = YB_NAs)
df.loc[mask, "Year Built"] = fill

In [ ]:
df.info()

In [ ]:
#double check distribution of Year Built
plt.hist(df["Year Built"], bins = 100)
plt.title("Property Distribution by Year Built", fontsize = 15)
plt.ylabel("Number of Properties", fontsize = 12)
plt.xlabel("Year Built", fontsize = 12);

In [ ]:
#check missing values in Lot Size and Date Sold
print(len(df[df["Lot Size"] == "—"])) #368 missing values in LotSize, make sense to be missing for condo

print(len(df[df["Date Sold"] == "—"]))
print(len(df[df["Date Sold"] == ""])) #0 missing values in Date Sold, should divide in month and year

#extract the year and month from Date Sold
df["year_sold"] = pd.DatetimeIndex(df["Date Sold"]).year
df["month_sold"] = pd.DatetimeIndex(df["Date Sold"]).month

In [ ]:
# #Does property price goes up or down the past year?
# plt.rcParams["figure.figsize"] = [6,6]
# plt.rcParams["figure.dpi"] = 200

# price_month = df.groupby(["year_sold", "month_sold"])["Price"].median()
# num_prop = df.groupby(["year_sold", "month_sold"])["Price"].count()

# plt.subplot(2,1,1)

# plt.plot(range(len(price_month)), price_month, linewidth = 3)
# plt.title("Median Property Price in the Past Year", fontsize = 15, weight = "bold")
# plt.ylabel("Median Property Price", fontsize = 12, weight = "bold")
# ytick_lab = ["1.5million", "1.6million", "1.7million", "1.8million", "1.9million"]
# plt.yticks(ticks = list(range(1500000, 2000000, 100000)), labels = ytick_lab)

# plt.subplot(2,1,2)
# #change color of every other bar
# barlist = plt.bar(range(len(num_prop)), num_prop)
# for bar in range(0, len(barlist), 2):
#     barlist[bar].set_color("lightblue")
# plt.title("Number of Property Sold in the Past Year", fontsize = 15, weight = "bold")
# plt.ylabel("Number of Property", fontsize = 12, weight = "bold")
# plt.xlabel("Jan 2020 to Jan 2021", fontsize = 12, weight = "bold")
# xtick_lab = ["Jan 20", "Feb 20", "Mar 20", "Apr 20", "May 20", "Jun 20", 
#              "Jul 20", "Aug 20", "Sep 20", "Oct 20", "Nov 20", "Dec 20", "Jan 21"]
# plt.xticks(range(len(price_month)), xtick_lab, rotation = 45)
# plt.tight_layout();
# plt.savefig("project2_images/median_price_past_year.png");

In [ ]:
#Does property price goes up or down the past year?
price_month = df.groupby(["year_sold", "month_sold"])["Price"].median()
num_prop = df.groupby(["year_sold", "month_sold"])["Price"].count()

plt.rcParams["figure.figsize"] = [6,6]
plt.rcParams["figure.dpi"] = 200
fig, axes = plt.subplots(2, 1, sharex=True)

ax = axes[0]

ax.plot(range(len(price_month)), price_month, linewidth = 3)
ax.set_title("Median Property Price in the Past Year", fontsize = 15, weight = "bold")
ax.set_ylabel("Median Property Price", fontsize = 12, weight = "bold")
ax.set_yticks(ticks = list(range(1500000, 2000000, 100000)))
ax.set_yticklabels(["1.4million", "1.5million", "1.6million", "1.7million", "1.8million"])


ax2 = axes[1]
#change color of every other bar
barlist = ax2.bar(range(len(num_prop)), num_prop)
for bar in range(0, len(barlist), 2):
    barlist[bar].set_color("lightblue")
ax2.set_title("Number of Property Sold in the Past Year", fontsize = 15, weight = "bold")
ax2.set_ylabel("Number of Property", fontsize = 12, weight = "bold")

plt.xlabel("Jan 2020 to Jan 2021", fontsize = 12, weight = "bold")
xtick_lab = ["Jan 20", "Feb 20", "Mar 20", "Apr 20", "May 20", "Jun 20", 
             "Jul 20", "Aug 20", "Sep 20", "Oct 20", "Nov 20", "Dec 20", "Jan 21"]
plt.xticks(range(len(price_month)), xtick_lab, rotation = 45)

plt.tight_layout();
plt.savefig("project2_images/median_price_past_year.png");

In [ ]:
#check missing values in Zip Code
print(len(df[df["Zip Code"] == "—"]))
print(len(df[df["Zip Code"] == ""])) #0 missing values in Zip Code, need to extract Zip Code from of the string

#extrac zip code from the string
df["Zip Code"] = [zip.split("-")[-1] for zip in df["Zip Code"]]
df["Zip Code"].value_counts()

In [ ]:
#one zip code (one listing) maps to Daly City, delete
df.drop(df[df["Zip Code"] == "94014"].index, inplace = True)

In [ ]:
df.info()

In [ ]:
df.to_pickle("/Users/sarazzzz/Desktop/Metis/CAMP/Metis_project2/prop_df_cleaned")